In [1]:
%load_ext autotime

time: 0 ns


## 第一部分：了解 nn.Module的基本操作

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

time: 2.45 s


In [3]:
model = models.resnet18()

time: 171 ms


### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [6]:
for name, param in model.named_parameters():
    print(name, ': ', param.shape)

conv1.weight :  torch.Size([64, 3, 7, 7])
bn1.weight :  torch.Size([64])
bn1.bias :  torch.Size([64])
layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight :  torch.Size([64])
layer1.0.bn1.bias :  torch.Size([64])
layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight :  torch.Size([64])
layer1.0.bn2.bias :  torch.Size([64])
layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight :  torch.Size([64])
layer1.1.bn1.bias :  torch.Size([64])
layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight :  torch.Size([64])
layer1.1.bn2.bias :  torch.Size([64])
layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight :  torch.Size([128])
layer2.0.bn1.bias :  torch.Size([128])
layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight :  torch.Size([128])
layer2.0.bn2.bias :  torch.Size([128])
layer2.0.downsample.0.weight :  torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight :  torch.Size([128])
l

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [7]:
input_ = torch.randn(1, 3, 128, 128)

time: 0 ns


In [9]:
for param in model.parameters():
    param.requires_grad = False

time: 0 ns


In [10]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])
time: 94 ms


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [14]:
input_ = torch.randn(10,12)
## 示範
Linear = 
Linear2 = nn.Linear(10,10)
Linear3 = nn.Linear(10,3)
BN = nn.BatchNorm1d(10)


time: 0 ns


In [17]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer = torch.nn.Sequential()
        
        self.layer.add_module('Linear-1', nn.Linear(12,10))
        self.layer.add_module('BatchNorm1d-1', nn.BatchNorm1d(10))
        for i in range(2,12):
            self.layer.add_module(f'Linear-{i}', nn.Linear(10,10))
            self.layer.add_module(f'BatchNorm1d-{i}', nn.BatchNorm1d(10))

        self.layer.add_module(f'Linear-{i+1}',nn.Linear(10,3))

    def forward(self, x):
        for model in self.layer:
            x = model(x)
        return x

time: 0 ns


In [18]:
model = Model()

time: 0 ns


In [19]:
model

Model(
  (layer): Sequential(
    (Linear-1): Linear(in_features=12, out_features=10, bias=True)
    (BatchNorm1d-1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Linear-2): Linear(in_features=10, out_features=10, bias=True)
    (BatchNorm1d-2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Linear-3): Linear(in_features=10, out_features=10, bias=True)
    (BatchNorm1d-3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Linear-4): Linear(in_features=10, out_features=10, bias=True)
    (BatchNorm1d-4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Linear-5): Linear(in_features=10, out_features=10, bias=True)
    (BatchNorm1d-5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Linear-6): Linear(in_features=10, out_features=10, bias=True)
    (BatchNorm1d-6): BatchNorm1d(10, eps=1e-05, momentum=0.1, a

time: 16 ms


In [20]:
input_ = torch.randn(10,12)
outupt = model(input_)

time: 31 ms


In [21]:
outupt

tensor([[ 0.3030,  0.0684,  0.0088],
        [-0.7783,  0.3395, -0.3993],
        [-0.1957,  0.2630, -0.4544],
        [ 1.3816, -0.2868,  0.6722],
        [-1.0311,  0.6345, -0.8778],
        [-1.6237,  0.7222, -1.0283],
        [-0.2541,  0.3039, -0.3331],
        [-0.3585,  0.2606, -0.5201],
        [-0.1615,  0.1695, -0.0398],
        [ 0.1332,  0.0415,  0.0664]], grad_fn=<AddmmBackward>)

time: 16 ms
